# ECE 4194.04 Final Project - Citation Thing
  __Authors:__  Omar Mansour, Shreyas Murlaidharan, Ryan Weiper

## Part 0: Data pre-processing

The dataset is in a bastardized JSON format. We first iterate over it to put it into a pandas dataframe

In [7]:
import pandas as pd
import numpy as np

# Open the data
#data_str = open('acm1st10000.txt',errors="ignore").read()     # small data
data_str = open('data/acm.v9/acm.txt',encoding="utf8").read()  # full dataset

data_list = data_str.splitlines() # split by line breaks

# Create the dataframe
    # title: (string -- should be bag of words vector)
    # authors: (list)
    # year: publishing year (int)
    # venue: journal the paper was published in (string)
    # ind: numeric label/index of this paper (int)
    # ref_indces: identification index of the papers that referenced this paper (string)
    # abstract: (string -- should be bag of words vector ?)
names = ['title', 'authors', 'year', 'venue', 'abstract','citations']

title = "NULL"
authors = "NULL"
year = "NULL"
venue = "NULL"
index = 0
abstract = "NULL"
citations = 0

df_list = []

for i in range(len(data_list)):
    if(i%10**6 == 0):  # progress update 
        print(i)
    prefix = data_list[i][0:2]
    if prefix == "#*": # title
        title = data_list[i][2:]
        continue
    if prefix == "#@": #author list
        authors = data_list[i][2:]
        continue
    if prefix == "#t": # year
        year = data_list[i][2:]
        continue
    if prefix == "#c": #journal
        venue = data_list[i][2:]
        continue
    if prefix == "#!": #abstract
        abstract = data_list[i][2:]
    if prefix == "#%": #paper citation
        citations += 1
    if data_list[i] == "": #check for the empty string (new article)       
        if year == "2015":
            df_list.append([title, authors, year, venue, abstract,citations])
            # print(index,citations) #debug
            title = "NULL"
            authors = "NULL"
            year = "NULL"
            venue = "NULL"
            abstract = "NULL"
            
            index += 1
        citations = 0

print("Done")

df = pd.DataFrame(columns=names)
df = pd.DataFrame.from_records(df_list, columns=names)
df.head(10)

    

# T0-DO:
# -----------------------------------------------------------
# Force strings in dataframe that we want as ints to be ints
# Convert the strings in dataframe to bag of words vectors 
# Make a dataframe of just 2015 papers

# Account for no citations in a dataframe (if citation column is NaN, make it 0)
# Account for no abstract (if abstract column is NaN, remove the row completely)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
Done


,title,authors,year,venue,abstract,citations
0,Finding an Initial Basic Feasible Solution for...,"Mehdi Toloo, Atefeh Masoumzadeh, Mona Barat",2015,Computational Economics,"Nowadays, algorithms and computer programs, wh...",3
1,Tag line generating system using knowledge ext...,"Hiroaki Yamane, Masafumi Hagiwara",2015,AI & Society,This paper proposes a tag line generating syst...,2
2,Computational Economics,NULL,2015,Computational Economics,NULL,0
3,A social contract for virtual institutions,Daniel Memmi,2015,AI & Society,"Computer-mediated social groups, often known a...",6
4,AI & Society,NULL,2015,AI & Society,NULL,0
5,Solving Dynamic Programming Problems on a Comp...,"Yongyang Cai, Kenneth L. Judd, Greg Thain, Ste...",2015,Computational Economics,We implement a dynamic programming algorithm o...,8
6,ESIS2: Information Value Estimator for Credit ...,Martin Řezáč,2015,Computational Economics,Information value is widely used to assess dis...,5
7,Social media analytics: a survey of techniques...,"Bogdan Batrinca, Philip C. Treleaven",2015,AI & Society,This paper is written for (social science) res...,9
8,Carbon Price Analysis Using Empirical Mode Dec...,"Bangzhu Zhu, Ping Wang, Julien Chevallier, Yim...",2015,Computational Economics,Mastering the underlying characteristics of ca...,4
9,Crime detection and criminal identification in...,"Devendra Kumar Tayal, Arti Jain, Surbhi Arora,...",2015,AI & Society,"In the current paper, we propose an approach f...",9


## Part 1: Aggregate and cluster?

blah blah

In [15]:
#print(df.head(5))
#print(newdf)
list_2k15_X=df['title'].tolist()
list_2k15_y=df['citations'].tolist()
#print(list_2k15_Y)


#print list of journals
list_journals = df['venue'].unique()
list_authors  = df['authors'].unique()
print("There are %s unique journal names :)" % len(list_journals))
print("There are %s unique author names :)" % len(list_authors))



There are 1099 unique journal names :)
There are 23239 unique author names :)


#### Create author score

In [51]:
# make a dictionary or author name and citations
    # key is author name
    # value is list: [number of citations,number papers]
        # (avg citation count can be computer as author_dict["name"][0]/author_dict["name"][1])
author_dict = {}
    
# loop over data frame
for i in range(len(df.index)): 
    for author in df.iloc[i][1].split(","):
        a = author.strip()
        cit = df.iloc[i][5]   # citation count
        if a in author_dict:   #author already is in dictionary
            author_dict[a] = [author_dict[a][0]+cit,author_dict[a][1]+1]
        else:                 # author not in dictionary
            author_dict[a] = [cit,1] 

In [52]:
author_avg_citations_dict = {}
for key in author_dict:
    author_avg_citations_dict[key] = author_dict[key][0]/author_dict[key][1]

# test code
print("Mehdi Toloo avg citations" author_avg_citations_dict['Mehdi Toloo']) #should be 5/3


# Create a list of author scores to replace the df['authors'] column
# author score is simply the mean of the avg citations counts for each author
authorscore_list = []
for i in range(len(df.index)): 
    score = 0
    authors = df.iloc[i][1].split(",")
    for author in authors:
        score += author_avg_citations_dict[author.strip()]
    authorscore_list.append(score/len(authors))    


SyntaxError: invalid syntax (<ipython-input-52-b6b7228049ef>, line 6)

In [49]:
print(authorscore_list[0:100])
  
# Add column of author score to df
df['author score'] = authorscore_list

[2.555555555555556, 1.0, 0.47252182845403184, 3.0, 0.47252182845403184, 7.125, 5.0, 9.0, 3.375, 9.0, 7.0, 2.0, 1.0, 5.0, 15.166666666666666, 0.47252182845403184, 7.0, 22.0, 1.0, 11.833333333333334, 11.541666666666668, 3.0, 8.166666666666666, 13.7, 24.0, 19.25, 2.0, 1.0, 5.772321428571429, 26.0, 27.833333333333332, 1.75, 9.083333333333334, 26.375, 1.6666666666666665, 0.47252182845403184, 6.0625, 0.47252182845403184, 0.47252182845403184, 1.0, 0.47252182845403184, 1.0, 2.0, 2.0, 0.47252182845403184, 5.0, 2.0, 3.0, 0.47252182845403184, 1.0, 4.0, 0.47252182845403184, 1.0, 1.0, 1.0, 37.0, 25.944444444444443, 45.833333333333336, 1.0, 2.0, 13.020833333333334, 73.0, 22.75, 18.0, 2.0, 25.875, 8.0, 6.625, 16.0, 8.571428571428571, 35.833333333333336, 17.598484848484848, 2.5, 16.113636363636363, 17.0, 7.466666666666666, 46.0, 8.0, 8.0, 11.833333333333334, 23.0, 11.875, 33.0, 14.766666666666666, 13.0, 11.0, 15.0, 11.833333333333334, 16.738311688311686, 3.0, 14.333333333333334, 19.333333333333332, 9.

In [50]:
df.head()

,title,authors,year,venue,abstract,citations,author score
0,Finding an Initial Basic Feasible Solution for...,"Mehdi Toloo, Atefeh Masoumzadeh, Mona Barat",2015,Computational Economics,"Nowadays, algorithms and computer programs, wh...",3,2.555556
1,Tag line generating system using knowledge ext...,"Hiroaki Yamane, Masafumi Hagiwara",2015,AI & Society,This paper proposes a tag line generating syst...,2,1.000000
2,Computational Economics,NULL,2015,Computational Economics,NULL,0,0.472522
3,A social contract for virtual institutions,Daniel Memmi,2015,AI & Society,"Computer-mediated social groups, often known a...",6,3.000000
4,AI & Society,NULL,2015,AI & Society,NULL,0,0.472522


In [9]:
list_journals.sort()
for i in range(1000):
    print(list_journals[i])

10th ACM Symposium on Information, Computer and Communications Security
11th ACM SIGPLAN/SIGOPS International Conference on Virtual Execution Environments
13th Annual IEEE/ACM International Symposium on Code Generation and Optimization
14th International Conference on Modularity
16th Annual International Digital Government Research Conference
18th International Conference on Hybrid Systems: Computation and Control (part of CPS Week)
19th International Conference on Evaluation and Assessment in Software Engineering
2015 Computers and People Research Conference
2015 International Conference on Advanced Research in Computer Science Engineering & Technology (ICARCSET 2015)
2015 International Conference on Supercomputing
2015 Software Infrastructure for Sustained Innovation (SI2) Principal Investigators Workshop
20th ACM SIGPLAN Symposium on Principles and Practice of Parallel Programming
20th ACM Symposium on Access Control Models and Technologies
24th International World Wide Web Conferen

In [10]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Omar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
ntot = len(list_2k15_X)
print(ntot)
from sklearn.feature_extraction.text import CountVectorizer
set_to_add=set(['using','based','system','systems','study'])
stop_words_NLTK = set(stopwords.words('english'))
total_stopwords=stop_words_NLTK|set_to_add
BOWvectorizer = CountVectorizer(stop_words=total_stopwords, min_df=0.01, max_df=0.99, max_features=50)
X_bow = BOWvectorizer.fit_transform(list_2k15_X).toarray()
y = list_2k15_y
print(BOWvectorizer.get_feature_names())


26921
['acm', 'adaptive', 'algorithm', 'analysis', 'application', 'applications', 'approach', 'computer', 'computing', 'conference', 'control', 'data', 'design', 'detection', 'dynamic', 'editorial', 'efficient', 'energy', 'framework', 'fuzzy', 'high', 'human', 'image', 'information', 'international', 'journal', 'learning', 'management', 'method', 'mobile', 'model', 'modeling', 'models', 'multi', 'network', 'networks', 'new', 'online', 'optimization', 'performance', 'problem', 'problems', 'proceedings', 'security', 'sensor', 'social', 'software', 'time', 'web', 'wireless']


## Part 2: Model and Predictions

heres the actual ML

In [14]:
ntr = ntot // 2
nts = ntot - ntr

Xtr = X_bow[:ntr,:]
ytr = y[:ntr]
Xts = X_bow[ntr:,:]
yts = y[ntr:]


print(X_bow.shape)
print(X_bow[0])

import sklearn.linear_model

# TODO

# Create linear regression object
regr = sklearn.linear_model.LinearRegression()

# Fit the model
regr.fit(Xtr,ytr)



# predict
yts_hat = regr.predict(Xts)

for i in range(100):
    print(yts[i],yts_hat[i])


(26921, 50)
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
1 1.6704258116123882
1 1.9196747179738862
1 2.305929185192454
0 1.6704258116123882
1 2.0957475101249714
1 1.6704258116123882
1 2.1881446518986225
1 1.6704258116123882
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
1 3.10191429093077
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
1 1.6002133610497957
0 1.3270882005020035
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
0 0.20082588837957482
1 1.278246963045017
0 0.20082588837957482
1 2.5325731152316475
0 0.20082588837957482
0 0.20082588837957482
1 1.6329708894467212
0 0.20082588837957482
0 2.0957475101249714
0 0.20082588837957482
0 0.20082588837957482
1 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDFVectorizer=TfidfVectorizer(stop_words=stop_words_NLTK, min_df=0.01, max_df=0.9, max_features=30)
X = TFIDFVectorizer.fit_transform(list_2k15_X)
print(TFIDFVectorizer.get_feature_names())
